In [ ]:
import time
from google.colab import files
import io
import pandas as pd
import requests
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Load the peptide data
file_path = '/content/combined_modified_peptide.tsv'
df_peptide = pd.read_csv(file_path, sep='\t')

# Load the Wollscheid list
with open('/content/surfaceome_accession_wollschied (2).txt', 'r') as file:
    wollscheid_set = set(file.read().splitlines())

# Filter out rows without necessary data in the peptide dataframe
df_peptide = df_peptide[df_peptide['Peptide Sequence'].notna() & df_peptide['Assigned Modifications'].notna() & df_peptide['Assigned Modifications'].str.strip() != '']

# Check the number of rows after filtering
#num_rows = len(df_peptide)
#print(f"The number of rows in the dataset after filtering: {num_rows}")

# Retry configuration
retry_strategy = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)

@lru_cache(maxsize=5000)
def get_protein_sequence(uniprot_id):
    """ Cached version to avoid repeated network requests for the same UniProt ID. """
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    response = http.get(url, timeout=10)
    if response.status_code == 200:
        return ''.join(response.text.split('\n')[1:])
    return None

# function to retrieve and check extracellular domains
def get_extracellular_domains(uniprot_id):
    """ Retrieves and checks for extracellular domains. """
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.txt"
    response = http.get(url, timeout=10)
    domains = []
    found_topo_dom = False
    if response.status_code == 200:
      lines = response.text.split('\n')
      for i in range(len(lines)):
        line = lines[i]
        if 'TOPO_DOM' in line:
            found_topo_dom = True
            if 'Extracellular' in next_line(lines, i+1):
                parts = line.split()
                if len(parts) > 2:
                    range_str = parts[2]
                    if '..' in range_str:
                      start, end = map(int, range_str.split('..'))
                      domains.append((start, end))
                    else:
                        print(f"Unexpected range format in line: {line}")
                else:
                        print(f"Unexpected format, not enough parts in line: {line}")
      if not found_topo_dom:
            return None
    return domains

def next_line(lines, index):
    """ Helper function to safely return the next line if it exists. """
    if index < len(lines):
        return lines[index]
    return ""

# Functions to find positions and check for modifications
def find_position_in_protein(full_sequence, peptide_sequence, amino_acid_position):
  index = full_sequence.find(peptide_sequence)
  return index + amino_acid_position if index != -1 else -1

def is_position_extracellular(position, domains):
  return any(start <= position <= end for start, end in domains)

def parse_modifications(mod_str):
  if isinstance(mod_str, float):
    mod_str = str(mod_str)
  return [int(m.split('(')[0][:-1]) for m in mod_str.split(', ') if '(' in m and m.split('(')[0][:-1].isdigit()]

# Function to process each row with additional Wollscheid check
def process_row(index, row, results):
  start_time = time.time()
  protein_sequence = get_protein_sequence(row['Protein ID'])
  is_wollscheid = 'Yes' if row['Protein ID'] in wollscheid_set else 'No'
  if protein_sequence:
    mod_positions = parse_modifications(row['Assigned Modifications'])
    domains = get_extracellular_domains(row['Protein ID'])
    row_results = []
    for mod_pos in mod_positions:
      pos_in_protein = find_position_in_protein(protein_sequence, row['Peptide Sequence'], mod_pos)
      if pos_in_protein != -1:
        if domains is None:
           extracellular_status = 'Not Available'
        else:
           extracellular_status = 'Yes' if is_position_extracellular(pos_in_protein, domains) else 'No'
        row_results.append((pos_in_protein, extracellular_status, is_wollscheid))
    results[index] = row_results
  else:
    results[index] = [('Protein sequence not found', 'Protein sequence not found', is_wollscheid)]
  print(f"Processed row {index} in {time.time() - start_time:.2f} seconds")

# Add new columns for results
df_peptide['Position in Protein'] = None
df_peptide['Is Extracellular'] = None
df_peptide['IsWollschied'] = None

# Use ThreadPoolExecutor to process rows concurrently
start_time = time.time()
results = {}
with ThreadPoolExecutor(max_workers=20) as executor:
  futures = [executor.submit(process_row, index, row, results) for index, row in df_peptide.iterrows()]
  for future in as_completed(futures):
    future.result()  # Ensure any exceptions are raised

# Update DataFrame with results
for index, row_results in results.items():
  df_peptide.at[index, 'Position in Protein'] = ', '.join(str(res[0]) for res in row_results if res[0] is not None)
  df_peptide.at[index, 'Is Extracellular'] = ', '.join(res[1] for res in row_results)
  df_peptide.at[index, 'IsWollschied'] = ', '.join(res[2] for res in row_results)

print(f"Processed all rows in {time.time() - start_time:.2f} seconds")

# Save the updated DataFrame to a new Excel file
updated_file_path = 'updated_peptide.xlsx'
df_peptide.to_excel(updated_file_path, index=False)
files.download(updated_file_path)

The number of rows in the dataset after filtering: 60211


Processed row 17 in 3.61 seconds
Processed row 14 in 3.65 seconds
Processed row 1 in 3.74 seconds
Processed row 11 in 3.68 secondsProcessed row 9 in 3.68 seconds
Processed row 5 in 3.71 seconds
Processed row 8 in 3.71 seconds



Processed row 3 in 4.30 secondsProcessed row 6 in 4.29 seconds
Processed row 16 in 4.29 seconds

Processed row 15 in 4.29 seconds
Processed row 7 in 4.31 seconds
Processed row 18 in 4.46 seconds
Processed row 13 in 4.48 seconds
Processed row 2 in 4.67 seconds
Processed row 4 in 4.68 seconds
Processed row 12 in 4.73 seconds
Processed row 19 in 4.72 seconds
Processed row 0 in 4.88 seconds
Processed row 10 in 4.94 seconds
Processed row 20 in 1.89 seconds
Processed row 25 in 1.82 seconds
Processed row 26 in 1.83 seconds
Processed row 22 in 1.87 secondsProcessed row 24 in 1.85 seconds

Processed row 23 in 1.87 seconds
Processed row 21 in 1.91 seconds
Processed row 27 in 1.41 seconds
Processed row 29 in 1.42 seconds
Processed row 30 in 1.42 seconds
Processed row 31 in 1.41 seconds
Processed row 28 in 1.43 seconds
Processed row 32 in 1.35 seconds
Processed row 33 in 1.39 seconds
Processed row 34 in 1.39 seconds
Processed row 35 in 1.41 seconds
Processed row 38 in 1.30 seconds
Processed row 37

Processed row 100 in 1.51 seconds
Processed row 92 in 2.03 seconds
Processed row 103 in 1.47 seconds
Processed row 102 in 1.55 seconds
Processed row 101 in 1.59 seconds


Processed row 99 in 1.97 seconds
Processed row 105 in 1.55 seconds
Processed row 109 in 1.49 seconds
Processed row 108 in 1.56 seconds
Processed row 107 in 1.61 seconds


...

Processed row 59991 in 0.40 seconds
Processed row 59990 in 0.42 seconds
Processed row 59992 in 0.41 seconds
Processed row 59988 in 0.45 seconds
Processed row 59983 in 0.57 seconds
Processed row 59989 in 0.44 seconds
Processed row 59977 in 0.85 seconds
Unexpected range format in line: FT   TOPO_DOM        86
Processed row 59978 in 0.85 seconds
Processed row 59993 in 0.43 seconds
Processed row 59994 in 0.46 seconds
Processed row 59975 in 0.92 seconds
Processed row 59995 in 0.47 seconds
Processed row 59996 in 0.43 seconds
Processed row 59997 in 0.43 seconds
Processed row 59998 in 0.42 seconds
Processed row 60000 in 0.40 seconds
Processed row 60001 in 0.40 seconds
Processed row 60002 in 0.42 seconds
Processed row 60003 in 0.41 seconds
Processed row 60006 in 0.40 seconds
Processed row 60004 in 0.42 seconds


Processed row 60007 in 0.42 seconds
Processed row 60005 in 0.46 seconds
Processed row 60009 in 0.46 seconds
Processed row 60011 in 0.46 seconds
Processed row 60010 in 0.47 seconds
Processed row 60014 in 0.44 seconds
Processed row 60016 in 0.42 seconds
Processed row 60017 in 0.42 seconds
Processed row 60018 in 0.45 seconds
Processed row 60019 in 0.42 seconds
Processed row 60020 in 0.42 seconds
Processed row 60008 in 0.75 seconds
Processed row 60021 in 0.44 seconds
Processed row 60023 in 0.42 seconds
Processed row 60022 in 0.43 seconds
Processed row 60024 in 0.45 seconds
Processed row 60025 in 0.44 seconds
Processed row 60015 in 0.81 seconds
Processed row 60027 in 0.42 seconds
Processed row 60026 in 0.45 seconds
Processed row 60028 in 0.45 seconds
Processed row 60030 in 0.42 seconds
Processed row 60031 in 0.42 seconds
Processed row 60013 in 0.94 seconds
Processed row 60012 in 0.98 seconds
Processed row 60033 in 0.44 seconds
Processed row 60034 in 0.44 seconds
Processed row 60035 in 0.42 

Processed row 60193 in 0.48 seconds
Processed row 60196 in 0.42 seconds
Processed row 60199 in 0.42 seconds
Processed row 60200 in 0.43 seconds
Processed row 60198 in 0.45 seconds


Processed row 60201 in 0.43 seconds
Processed row 60202 in 0.43 seconds
Processed row 60203 in 0.41 seconds
Processed row 60204 in 0.42 seconds
Processed row 60207 in 0.41 seconds
Processed row 60205 in 0.45 seconds
Processed row 60191 in 0.82 seconds
Processed row 60206 in 0.46 seconds
Processed row 60186 in 0.92 seconds
Processed row 60197 in 0.62 seconds
Processed row 60209 in 0.42 seconds
Processed row 60210 in 0.43 seconds
Processed row 60208 in 0.55 seconds


ValueError: invalid literal for int() with base 10: '?'